In [4]:
# pip install fitparse

from fitparse import FitFile
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Відкрити FIT файл
fitfile = FitFile('17003786880_ACTIVITY.fit')

# Ініціалізуємо список для зберігання даних
records = []

# Перебираємо записи у файлі
for record in fitfile.get_messages('record'):
    record_data = {}
    for record_field in record:
        record_data[record_field.name] = record_field.value
    records.append(record_data)

# Створюємо DataFrame з отриманих записів
df = pd.DataFrame(records)

df['time_only'] = df['timestamp'].dt.time
# Обчислення ковзного середнього зі вікном в 3 рядки (наприклад)
df['heart_rate_rolling_avg'] = df['heart_rate'].rolling(window=5).mean()


start_time = pd.to_datetime('04:08:00').time()
end_time = pd.to_datetime('04:15:00').time()

# Фільтруємо DataFrame за інтервалом часу
df_filtered = df[(df['time_only'] >= start_time) & (df['time_only'] <= end_time)]


dff = df_filtered[['time_only', 'heart_rate', 'heart_rate_rolling_avg']]

In [5]:
fig = go.Figure()

# Додавання першої лінії для heart_rate
fig.add_trace(go.Scatter(x=dff['time_only'], y=dff['heart_rate'], mode='lines', name='Heart Rate'))

# Додавання другої лінії для heart_rate_rolling_avg
fig.add_trace(go.Scatter(x=dff['time_only'], y=dff['heart_rate_rolling_avg'], mode='lines', name='Heart Rate Rolling Avg'))

# Вимкнення легенди
fig.update_layout(showlegend=False)

# Відображення графіку
fig.show()


In [6]:
fig = px.line(dff, x="time_only", y="heart_rate_rolling_avg")
fig.show()